In [4]:
import os
import sys
os.chdir('D:/repositories/News-Recommendation/Codes')
sys.path.append('D:/repositories/News-Recommendation/Codes')

import math
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
from torchtext.vocab import GloVe
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from utils.MIND import MIND_iter,MIND_map
from utils.utils import getLoss,getLabel,constructBasicDict,run_eval,run_train
from models.NRMS import NRMSModel

hparams = {
    'mode':'demo',
    'batch_size':5,
    'title_size':20,
    'his_size':20,
    'npratio':4,
    'dropout_p':0.2,
    'query_dim':200,
    'embedding_dim':300,
    'value_dim':16,
    'head_num':16,
    'kernel_num':11,
    'epochs':10,
    'metrics':'group_auc,ndcg@5,ndcg@10,mean_mrr',
    'gpu':'cuda:0',
    'attrs': ['title'],
}

news_file_train = 'D:/Data/MIND/MIND'+hparams['mode']+'_train/news.tsv'
news_file_test = 'D:/Data/MIND/MIND'+hparams['mode']+'_dev/news.tsv'
news_file_pair = (news_file_train,news_file_test)

behavior_file_train = 'D:/Data/MIND/MIND'+hparams['mode']+'_train/behaviors.tsv'
behavior_file_test = 'D:/Data/MIND/MIND'+hparams['mode']+'_dev/behaviors.tsv'
behavior_file_pair = (behavior_file_train,behavior_file_test)

save_path = 'models/model_params/NRMS_{}_{}'.format(hparams['mode'],hparams['epochs']) +'.model'

if not os.path.exists('data/dictionaries/vocab_{}_{}.pkl'.format(hparams['mode'],'_'.join(hparams['attrs']))):
    constructBasicDict(news_file_pair,behavior_file_pair,hparams['mode'],hparams['attrs'])

device = torch.device(hparams['gpu']) if torch.cuda.is_available() else torch.device("cpu")

dataset_train = MIND_map(hparams=hparams,news_file=news_file_train,behaviors_file=behavior_file_train)

dataset_test = MIND_iter(hparams=hparams,news_file=news_file_test,behaviors_file=behavior_file_test, mode='test')

vocab = dataset_train.vocab
embedding = GloVe(dim=300,cache='.vector_cache')
vocab.load_vectors(embedding)

loader_train = DataLoader(dataset_train,batch_size=hparams['batch_size'],shuffle=True,pin_memory=True,num_workers=3,drop_last=True)
loader_test = DataLoader(dataset_test,batch_size=hparams['batch_size'],pin_memory=True,num_workers=0,drop_last=True)

In [2]:
nrmsModel = NRMSModel(vocab=vocab,hparams=hparams).to(device)
nrmsModel.load_state_dict(torch.load(save_path))
nrmsModel.eval()

# nrmsModel = NRMSModel(vocab=vocab,hparams=hparams).to(device)
# nrmsModel.train()

NRMSModel(
  (softmax): Softmax(dim=-1)
  (ReLU): ReLU()
  (DropOut): Dropout(p=0.2, inplace=False)
  (queryProject_words): ModuleList(
    (0): Linear(in_features=300, out_features=300, bias=False)
    (1): Linear(in_features=300, out_features=300, bias=False)
    (2): Linear(in_features=300, out_features=300, bias=False)
    (3): Linear(in_features=300, out_features=300, bias=False)
    (4): Linear(in_features=300, out_features=300, bias=False)
    (5): Linear(in_features=300, out_features=300, bias=False)
    (6): Linear(in_features=300, out_features=300, bias=False)
    (7): Linear(in_features=300, out_features=300, bias=False)
    (8): Linear(in_features=300, out_features=300, bias=False)
    (9): Linear(in_features=300, out_features=300, bias=False)
    (10): Linear(in_features=300, out_features=300, bias=False)
    (11): Linear(in_features=300, out_features=300, bias=False)
    (12): Linear(in_features=300, out_features=300, bias=False)
    (13): Linear(in_features=300, out_feat

In [3]:
if nrmsModel.training:
    print("training...")
    loss_func = getLoss(nrmsModel)
    optimizer = optim.Adam(nrmsModel.parameters(),lr=0.001)
    nrmsModel = run_train(nrmsModel,loader_train,optimizer,loss_func,epochs=hparams['epochs'], interval=10)

training...
epoch 0 , step 390 , loss: 1.5753: : 400it [00:36, 11.06it/s]
epoch 1 , step 390 , loss: 1.5107: : 400it [00:34, 11.54it/s]
epoch 2 , step 390 , loss: 1.4434: : 400it [00:35, 11.42it/s]
epoch 3 , step 390 , loss: 1.3614: : 400it [00:35, 11.17it/s]
epoch 4 , step 390 , loss: 1.3103: : 400it [00:35, 11.29it/s]
epoch 5 , step 390 , loss: 1.1887: : 400it [00:34, 11.52it/s]
epoch 6 , step 390 , loss: 1.0855: : 400it [00:34, 11.43it/s]
epoch 7 , step 390 , loss: 0.9370: : 400it [00:35, 11.35it/s]
epoch 8 , step 390 , loss: 0.8366: : 400it [00:35, 11.30it/s]
epoch 9 , step 390 , loss: 0.6243: : 400it [00:34, 11.43it/s]


In [5]:
print("evaluating...")
nrmsModel.eval()
nrmsModel.cdd_size = 1
run_eval(nrmsModel,loader_test)

0it [00:00, ?it/s]evaluating...
2191it [01:26, 25.34it/s]


{'group_auc': 0.5452, 'ndcg@5': 0.2576, 'ndcg@10': 0.3175, 'mean_mrr': 0.239}

In [9]:
nrmsModel.cdd_size = 5
torch.save(nrmsModel.state_dict(), save_path)